Question 1: OLS Regression – Growth [25 points]
Using the Growth data, carry out the following exercises. Exclude the data for Malta.

a. [5] Construct a table that shows the sample mean, standard deviation,
and minimum and maximum values for the series Growth, TradeShare, YearsSchool, Oil, Rev_Coups, Assassinations, and RGDP60. Include the appropriate units for all entries.

In [ ]:
import pandas as pd

# Load the data, excluding Malta
df = pd.read_stata('data/Growth.dta')

In [4]:
df

,country_name,growth,oil,rgdp60,tradeshare,yearsschool,rev_coups,assasinations
0,India,1.915168,0.0,765.999817,0.140502,1.45,0.133333,0.866667
1,Argentina,0.617645,0.0,4462.001465,0.156623,4.99,0.933333,1.933333
2,Japan,4.304759,0.0,2953.999512,0.157703,6.71,0.000000,0.200000
3,Brazil,2.930097,0.0,1783.999878,0.160405,2.89,0.100000,0.100000
4,United States,1.712265,0.0,9895.003906,0.160815,8.66,0.000000,0.433333
...,...,...,...,...,...,...,...,...
60,Cyprus,5.384184,0.0,2037.000366,0.979355,4.29,0.100000,0.166667
61,Malaysia,4.114544,0.0,1420.000244,1.105364,2.34,0.033333,0.033333
62,Belgium,2.651335,0.0,5495.001953,1.115917,7.46,0.000000,0.000000
63,Mauritius,3.024178,0.0,2861.999268,1.127937,2.44,0.000000,0.000000


In [9]:
import pandas as pd
df_nomalta = df[df['country_name'] != 'Malta']

# Correct column names to match the DataFrame
cols = ['growth', 'tradeshare', 'yearsschool', 'oil', 'rev_coups', 'assasinations', 'rgdp60']

# Compute summary statistics
summary = df_nomalta[cols].agg(['mean', 'std', 'min', 'max']).T

# Optionally add units for interpretation (as a column or inline comments)
units = {
    'growth': 'percent per year',
    'tradeshare': 'fraction',
    'yearsschool': 'years',
    'oil': 'indicator (1/0)',
    'rev_coups': 'number',
    'assasinations': 'number per million',
    'rgdp60': 'US dollars (1960)'
}

summary['units'] = summary.index.map(units)

summary

,mean,std,min,max,units
growth,1.869120,1.816189,-2.811944,7.156855,percent per year
tradeshare,0.542392,0.228333,0.140502,1.127937,fraction
yearsschool,3.959219,2.553465,0.200000,10.070000,years
oil,0.000000,0.000000,0.000000,0.000000,indicator (1/0)
rev_coups,0.170067,0.225456,0.000000,0.970370,number
assasinations,0.281901,0.494159,0.000000,2.466667,number per million
rgdp60,3130.812500,2522.978516,366.999939,9895.003906,US dollars (1960)


b. Run a regression of Growth on TradeShare, YearsSchool, Rev_Coups, Assassinations,
and RGDP60. What is the value of the coefficient on Rev_Coups? Interpret the value of
this coefficient. Is it large or small in a real-world sense?

In [11]:
import statsmodels.api as sm

# 독립 변수와 종속 변수 준비
X = df_nomalta[['tradeshare', 'yearsschool', 'rev_coups', 'assasinations', 'rgdp60']]
X = sm.add_constant(X)
y = df_nomalta['growth']

# 회귀분석 실행
model = sm.OLS(y, X).fit()

# 결과 출력
print(model.summary())

# rev_coups 계수 추출 및 출력
rev_coups_coef = model.params['rev_coups']
print(f"\nCoefficient on rev_coups: {rev_coups_coef:.4f}")

# 해석을 한국어로 출력
if abs(rev_coups_coef) >= 0.5:
    size_comment = "실질적으로 매우 큰 값입니다."
elif abs(rev_coups_coef) >= 0.1:
    size_comment = "실질적으로 중간 정도의 값입니다."
else:
    size_comment = "실질적으로 작은 값입니다."

ko_message = (
    f"Rev_Coups 변수의 계수는 {rev_coups_coef:.4f}입니다. "
    "이는 다른 변수들이 고정된 상태에서, 정변 쿠데타(Rev_Coups)가 1 증가할 때 "
    f"경제 성장률(growth)이 평균적으로 {rev_coups_coef:.4f} 포인트 변화함을 의미합니다. "
    f"이 계수는 {size_comment}"
)
print("\n"+ko_message)


                            OLS Regression Results                            
Dep. Variable:                 growth   R-squared:                       0.291
Model:                            OLS   Adj. R-squared:                  0.230
Method:                 Least Squares   F-statistic:                     4.764
Date:                Wed, 12 Nov 2025   Prob (F-statistic):            0.00103
Time:                        19:43:04   Log-Likelihood:                -117.49
No. Observations:                  64   AIC:                             247.0
Df Residuals:                      58   BIC:                             259.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.6269      0.783      0.801

c. Use the regression to predict the average annual growth rate for a country that has
average values for all regressors.

In [12]:
# 평균값으로 이루어진 새로운 관측치를 생성
mean_values = df_nomalta[['tradeshare', 'yearsschool', 'rev_coups', 'assasinations', 'rgdp60']].mean()
mean_values_with_const = [1] + mean_values.tolist()  # const 포함

# 예측 (평균값 대입)
pred_growth = model.predict([mean_values_with_const])[0]
print(f"Predicted average annual growth rate (using regressor means): {pred_growth:.4f}")


Predicted average annual growth rate (using regressor means): 1.8691


d. Repeat (c), but now assume that the country’s value for TradeShare is one standard
deviation above the mean.

In [14]:
# 평균값 벡터를 복사하여 TradeShare만 한 표준편차 더한 값으로 변경
mean_vals_sd = mean_values.copy()
trade_share_mean = mean_values['tradeshare']
trade_share_std = df_nomalta['tradeshare'].std()
mean_vals_sd['tradeshare'] = trade_share_mean + trade_share_std

mean_vals_sd_with_const = [1] + mean_vals_sd.tolist()  # const 포함

# 예측
pred_growth_sd = model.predict([mean_vals_sd_with_const])[0]
print(f"Predicted average annual growth rate (TradeShare = mean + 1 sigma): {pred_growth_sd:.4f}")


Predicted average annual growth rate (TradeShare = mean + 1 std): 2.1753


e. Why is Oil omitted from the regression? What would happen if it were included?

In [15]:
# Oil 변수는 데이터셋에서 모두 0의 값을 가지므로, 분산이 0이고 상수항과 완전히 공선(linearly dependent)합니다.
# 이런 경우 회귀식에 포함시키면 완벽한 다중공선성(perfect multicollinearity)로 인해 statsmodels가 자동으로 해당 변수를 제외(omit)합니다.
# 만약 강제로 포함하려 한다면 회귀모형이 추정되지 않거나, 결과에서 자동으로 제거되어 나타날 것입니다.


Question 2: Instrumental Variables – Returns to Schooling [40 points]
In this exercise, you will follow Card (1993) to estimate the returns to schooling, which is
a classic question in labor economics. It is generally accepted that schooling is positively
associated with earnings, but measuring the causal relationship between them is not
straightforward due to the endogeneity in schooling. To obtain a plausibly exogenous variation in
schooling, Card (1993) uses geographic proximity to colleges as an instrument for schooling. Use
the stata file, Card.dta.
To be more concrete, consider the following causal model:
log 𝑌 = 𝛼 + 𝛿𝑠 + 𝛾𝑋 + 𝜀
where 𝑌 is earnings, 𝑠 is years of schooling, 𝑋 is an (exogenous) covariate matrix, and 𝜀 is an
individual-level idiosyncratic shock. The OLS estimate of 𝛿 will not be consistent for the causal
effect because 𝑠 is endogenous. For example, 𝜀 may include unobserved characteristics, such as
an individual’s ability, which may correlate with both 𝑠 and 𝑌. Thus, Card (1993) proposes using
𝑧, an indicator for having a 4-year college in one’s county, as an instrumental variable for 𝑠.

a) [10] In order for 𝑧 to be a legitimate instrument variable, it must be a strong predictor of 𝑠.
Conceptually, do you think living close to a 4-year college in one’s county is a good predictor
of one’s schooling? Why or why not?

In [22]:
# Conceptually assessing whether living close to a 4-year college is a good predictor of years of schooling

# In general, proximity to a 4-year college may increase the likelihood that an individual pursues further education
# after high school due to reduced costs (such as moving or commuting) and increased accessibility. 
# Especially for students from lower-income families, local college availability often plays an important role 
# in the decision to continue schooling. However, other factors such as family preferences, individual ability, 
# and the value placed on higher education can also influence educational attainment, meaning proximity is not 
# a perfect predictor, but it is plausibly correlated with schooling.

conceptual_iv_predictor = (
    "Living close to a 4-year college is likely a good predictor of one's years of schooling. "
    "Proximity lowers costs and increases accessibility, making attending college more feasible, "
    "particularly for students from families with financial constraints. However, it is not a perfect predictor "
    "because other factors (like personal motivation, academic ability, or family expectations) also affect schooling."
)

print(conceptual_iv_predictor)


Living close to a 4-year college is likely a good predictor of one's years of schooling. Proximity lowers costs and increases accessibility, making attending college more feasible, particularly for students from families with financial constraints. However, it is not a perfect predictor because other factors (like personal motivation, academic ability, or family expectations) also affect schooling.


b) [10] For the exclusion restriction to be valid, living close to a 4-year college in one’s county
must not affect earnings directly. In other words, schooling must be the only channel through
which geographic proximity can affect one’s earnings. Do you think it is valid? Why or why
not?

Assuming that the exclusion restriction holds, let us estimate the IV regression equation using data.
Card (1993) uses the Young Men Cohort of the 1966 National Longitudinal Survey. The data file
is card.dta. Here are its key variables:
• wage: earnings
• educ: years of schooling
• exper: potential work experience (age - educ - 6)
2
• black: an indicator for the black population
• south: an indicator for living in a Southern state
• married: an indicator for being married
• smsa: an indicator for living in a metropolitan area of a Southern state
• nearc4: an indicator for having a 4-year college in the same county

In [24]:

import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

# Load the Card (1993) data
card = pd.read_stata("data/Card.dta")
print(card.head().to_string())

    id  nearc2  nearc4  educ   age  fatheduc  motheduc    weight  momdad14  sinmom14  step14  reg661  reg662  reg663  reg664  reg665  reg666  reg667  reg668  reg669  south66  black  smsa  south  smsa66   wage  enroll   KWW     IQ  married  libcrd14  exper     lwage  expersq   u_lwage  lwage_hat    u_educ
0  2.0     0.0     0.0   7.0  29.0       NaN       NaN  158413.0       1.0       0.0     0.0     1.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0      0.0    1.0   1.0    0.0     1.0  548.0     0.0  15.0    NaN      1.0       0.0   16.0  6.306275    256.0  0.198047   6.108228 -3.067539
1  3.0     0.0     0.0  12.0  27.0       8.0       8.0  380166.0       1.0       0.0     0.0     1.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0      0.0    0.0   1.0    0.0     1.0  481.0     0.0  35.0   93.0      1.0       1.0    9.0  6.175867     81.0 -0.211160   6.387027 -1.727562
2  4.0     0.0     0.0  12.0  34.0      14.0      12.0  367470.0       1.0       0

In [25]:
card[["wage", "educ", "exper", "black", "south", "married", "smsa", "nearc4"]].describe()


,wage,educ,exper,black,south,married,smsa,nearc4
count,3010.000000,3010.000000,3010.000000,3010.000000,3010.000000,3003.000000,3010.000000,3010.000000
mean,577.282410,13.263455,8.856146,0.233555,0.403654,2.271395,0.712957,0.682060
std,262.958313,2.676913,4.141672,0.423162,0.490711,2.066823,0.452457,0.465753
min,100.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,394.250000,12.000000,6.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,537.500000,13.000000,8.000000,0.000000,0.000000,1.000000,1.000000,1.000000
75%,708.750000,16.000000,11.000000,0.000000,1.000000,4.000000,1.000000,1.000000
max,2404.000000,18.000000,23.000000,1.000000,1.000000,6.000000,1.000000,1.000000


In [28]:
# Create log(wage) variable for later use, making sure to handle nonpositive wages as NaN
card["log_wage"] = np.where(card["wage"] > 0, np.log(card["wage"]), np.nan)


c) [10] Estimate the OLS and IV regression (2SLS) models, in which the dependent variable is
log(wage), the endogenous variable is educ, instrumented by nearc4, and exogenous variables
are exper, black, south, married, and smsa. Summarize the key variables’ coefficients and their
robust standard errors following the table presented below (you don’t need to report the
coefficients of the control variables). Does the framework suffer the weak instrument problem?
How do you interpret the estimation results?

In [32]:
# OLS regression
ols_formula = "log_wage ~ educ + exper + black + south + married + smsa"
ols_model = smf.ols(formula=ols_formula, data=card).fit(cov_type='HC1')

# First stage: regress educ on instrument and controls
first_stage_formula = "educ ~ nearc4 + exper + black + south + married + smsa"
first_stage = smf.ols(formula=first_stage_formula, data=card).fit(cov_type='HC1')

# Add the fitted values to the card DataFrame (not necessary, but informative for diagnostics)
card['educ_hat'] = first_stage.fittedvalues

# IV regression using 2SLS with statsmodels
from statsmodels.sandbox.regression.gmm import IV2SLS

# Set up dependent, endogenous, exog, and instrument variables
y = card["log_wage"]
# ENDOGENOUS variable(s) to be instrumented: 'educ'
# EXOGENOUS variable(s): 'exper', 'black', 'south', 'married', 'smsa' (with constant)
# INSTRUMENT(s): 'nearc4' plus all exogenous variables (with constant)

X_exog = card[["exper", "black", "south", "married", "smsa"]]
X_exog = sm.add_constant(X_exog)                    # exogenous regressors with a constant
X_endog = card[["educ"]]                            # endogenous regressors
Z = card[["nearc4"]]
Z_iv = pd.concat([X_exog, Z], axis=1)               # instruments + exogenous regressors (with constant)

# Use correct arguments for IV2SLS; cov_type is NOT a valid keyword for .fit()
iv2sls_model = IV2SLS(y, X_endog, Z_iv).fit()

# Display summary of OLS and IV (2SLS) for key vars
print("OLS Regression Results (Robust SE):")
print(ols_model.summary().tables[1].as_text())

print("\nFirst Stage F-stat on nearc4 (weak instrument test):")
print(first_stage.summary().tables[2].as_text())

print("\nIV Regression Results (2SLS):")
iv_results_table = iv2sls_model.summary().tables[1].as_text()
print(iv_results_table)

# For reporting: get key coefficients and robust SEs from both models
print("\nKey results summary:")
print(f"OLS: coef(educ) = {ols_model.params['educ']:.4f}, robust SE = {ols_model.bse['educ']:.4f}")
print(f"IV : coef(educ) = {iv2sls_model.params['educ']:.4f}, robust SE = {iv2sls_model.bse['educ']:.4f}")
print("\nFirst-stage F-statistic for nearc4 (rule of thumb: < 10 is considered weak):")
print(f"F-statistic: {first_stage.f_test('nearc4=0').fvalue[0][0]:.2f}")


MissingDataError: exog contains inf or nans

d) [10] Studies on the returns to schooling often suggest that the returns on schooling measured
in OLS may be overstated because of positive correlation between earnings and ability (recall
that if you measure the effect of schooling on earnings with binary treatment variable (𝐷),
selection bias will be positive due to the correlation between earnings and ability). Is your
empirical result in line with the expectation? If not, what could be the reasons for the
discrepancy? For the full points, provide the reasons beyond the in-class discussions.

Question 3 [Stock & Watson E12.2]: IV & Panel Data – Violent Movie and Crimes [45 points]
Does viewing a violent movie lead to violent behavior? If so, the incidence of violent crimes, such
as assaults, should rise following the release of a violent movie that attracts many viewers.
Alternatively, movie viewing may substitute for other activities (such as alcohol consumption) that
lead to violent behavior, so that assaults should fall when more viewers are attracted to the cinema.
The stata file Movies.dta contains data on the number of assaults and movie attendance for 516
weekends from 1995 through 2004.1 A detailed description is given in Movies_Description.pdf
file. The data set includes weekend U.S. attendance for strongly violent movies (such as Hannibal),
mildly violent movies (such as Spider-Man), and nonviolent movies (such as Finding Nemo).The
data set also includes a count of the number of assaults for the same weekend in a subset of counties
in the United States. Finally, the data set includes indicators for year, month, whether the weekend
is a holiday, and various measures of the weather.

a) [5] Regress the logarithm of the number of assaults [ln_assaults = ln(assaults)] on the year
and month indicators. Is there evidence of seasonality in assaults? That is, do there tend to
be more assaults in some months than others? Explain.

b) [5] Regress total movie attendance (attend = attend_v + attend_m + attend_n) on the year
and month indicators. Is there evidence of seasonality in movie attendance? Explain.

c) [5] Regress ln_assaults on attend_v, attend_m, attend_n, the year and month indicators,
and the weather and holiday control variables available in the data set.

d) [5] Based on the regression in c), does viewing a strongly violent movie increase or decrease assaults? By how much? Is the estimated effect statistically significant?

e) Based on the regression in c), does attendance at strongly violent movies affect assaults
differently than attendance at moderately violent movies? Differently than attendance at
nonviolent movies?

f) This part is based on the regression in c). A strongly violent blockbuster movie is
released, and the weekend’s attendance at strongly violent movies increases by 6 million;
meanwhile, attendance falls by 2 million for moderately violent movies and by 1 million
for nonviolent movies. What is the predicted effect on assaults? Construct a 95%
confidence interval for the change in assaults.

g) [10] It is difficult to control for all the variables that affect assaults and that might be
correlated with movie attendance. For example, the effect of the weather on assaults and
movie attendance is only crudely approximated by the weather variables in the data set.
However, the data set does include a set of instruments—pr_attend_v, pr_attend_m, and
pr_ attend_n—that are correlated with attendance but are (arguably) uncorrelated with
weekend-specific factors (such as the weather) that affect both assaults and movie
attendance. These instruments use historical attendance patterns, not information on a
particular weekend, to predict a film’s attendance in a given weekend. For example, if a
film’s attendance is high in the second week of its release, then this can be used to predict
that its attendance was also high in the first week of its release. (The details of the
construction of these instruments are available in the Dahl and DellaVigna paper
referenced in footnote 5.) Run the regression from (b) (including year, month, holiday, and
weather controls) but now using pr_attend_v, pr_attend_m, and pr_attend_n as
instruments for attend_v, attend_m, and attend_n. Use this IV regression to answer (b)(i)–
(b)(iii).

h) [5] Based on your analysis, what do you conclude about the effect of violent movies on
violent behavior?

Question 4: Difference-in-Differences – ACA Medicaid Expansion [45 points]
In this exercise, you will measure the effect of the ACA Medicaid expansion on health
insurance coverage again, but exploiting panel data structure this time. The data file for the
exercise is acamedicaid2.dta. It contains annual data on uninsured rates and socioeconomic
variables at the county level between 2010 and 2017. Here are the key variables from the data set:
• pctui: uninsured rate
• pctui: uninsured rate
• tot_pop: total population (in 1,000)
• tot_pop: total population (in 1,000)
• black: share of blacks
• black: share of blacks
• hisp: share of Hispanics
• hisp: share of Hispanics
• children: share of 19-year-olds and younger
• children: share of 19-year-olds and younger
• elderly: share of 65-year-olds and older
• elderly: share of 65-year-olds and older
• unemployment: the unemployment rate
• unemployment: the unemployment rate
• pci: per capita income
• pci: per capita income
• exp: status of participating in the ACA Medicaid expansion
• exp: status of participating in the ACA Medicaid expansion
• year: year
• year: year
• statefips: state identifier
• statefips: state identifier
• countyfips: county identifier
• countyfips: county identifier
As discussed in the previous exercise, states could choose to participate in the expansion or not,
and they also could participate in different years. Most states participated in 2014, but a few states
participated in the expansion between 2014 and 2020. In this exercise, we will focus on measuring
the effect of the expansion in 2014 (e.g., a homogeneous treatment timing). In other words, the
states that opted into the expansion in 2014 are considered as treated, and the states that did not

participate before 2020 are considered as untreated. Remaining states (i.e., the late-participants)
are excluded. The table above lists the states in the three groups. The final data set is balanced as
a small number of counties lacking data in some years are excluded.
The main goal of this exercise is to estimate the effect of the ACA Medicaid expansion on
uninsured rates at the county level using a differences-in-differences (DID) framework.
Throughout the exercise, compute robust standard errors clustered by state, and use the population
of non-elderly adults (tot_pop × (1 − children − elderly)) as a weight when computing averages
and estimating regression models. Put your regression outputs only in your log file.

Treated: AR, AZ, CA, CO, CT, DC, DE, HI, IA, IL, KY, MA, MD, MI, MN, ND, NH, NJ, NM, NV, NY, OH, OR, RI, VT, WA, WV
Untreated: AL, FL, GA, ID, KS, ME, MO, MS, NC, NE, OK, SC, SD, TN, TX, UT, VA, WI, WY
Excluded: AK, IN, LA, MT, PA


a. Let’s first check the pre-trends of the outcome variable to assess the parallel trends
assumption. Make a time-series plot of the annual means of pctui between 2010 and 2017 for
the treatment and control groups. When computing means, use the weighted means weighted
by the population of non-elderly adults. Does the plot exhibit parallel trends between the two
groups in the pre-period?

b. Use regression to estimate the DID parameter. Without using controls, estimate the
parameters in the following specifications. Do they give you the same coefficients for the DID
parameter (𝛿)? How do you interpret the coefficient?
(1) [Classic DID] 𝑌_{cst} = 𝛼_{1} + 𝛽 * 𝑃𝑜𝑠𝑡_{t} + 𝛾 * 𝑇𝑟𝑒𝑎𝑡_{s} + 𝛿 * 𝑃𝑜𝑠𝑡_{t} * 𝑇𝑟𝑒𝑎𝑡_{s} + 𝜀_{cst}
(2) [Two-way FE (state)] 𝑌_{cst} = 𝛼_{2} + 𝜆_{s} + 𝜇_{t} + 𝛿_{2} * 𝐷_{cst} + 𝑒_{cst}
(3) [Two-way FE (county)] 𝑌_{cst} = 𝛼_{3} + 𝜙_{c} + 𝜌_{t} + 𝛿_{3} * 𝐷_{cst} + 𝜖_{cst}
where 𝑌_{cst}: pctui of county 𝑐 in state 𝑠 in year 𝑡;
𝑃𝑜𝑠𝑡_{t} : an indicator for years ≥ 2014;
𝑇𝑟𝑒𝑎𝑡_{s}: an indicator for participation states (exp = 1);
𝜆_{s}: state-fixed effects;
𝜇_{t}, 𝜌_{t}: year fixed effects;
𝜙_{c} : county fixed effects;
𝐷_{cst}: indicator whether county 𝑐 in state 𝑠 is treated in year 𝑡:
𝜀_{cst}, 𝑒_{cst}, 𝜖_{cst}: county-level mean-zero random shocks.

c. In the next analysis, the following time-varying county-level covariates (𝑋_{cst}) will be
included in the DID models: log(tot_pop), black, hisp, children, elderly, unemployment, and
pci. Are they “good” control variables in the sense that they are not the outcomes of the
treatment? In what cases will they be “bad” controls?

d. Estimate the models with the covariates included. Do the three specifications give you the
same DID coefficients? Why or why not? Think about this question in terms of the regression-
anatomy formula. Based on these results, what is your conclusion on the impact of the ACA
Medicaid expansion on uninsured rates?

e. Make an event study plot like Miller, Johnson, and Wherry (2021) by estimating the
following model.

&)%*
𝑌 "#$ = 𝑇𝑟𝑒𝑎𝑡# × I 𝛽(
(+&)%)
(,&)%'
𝟏(𝑡 = 𝑦) + 𝜆# + 𝜇$ + 𝑋!#$ + 𝑒"#$

Does the plot support the parallel trends assumption? Do you see any dynamics in the treatment
effect?

f. Check if your DID result is robust by including state-specific linear time trends
L∑ 𝟏(𝑦 = 𝑠) ⋅ 𝑡
( N in your DID specification. What is your conclusion?

g. Check if your DID result is robust by estimating the triple difference model in which you
interact the following three variables: 𝑃𝑜𝑠𝑡$, 𝑇𝑟𝑒𝑎𝑡#, and 𝐿𝑜𝑤-𝐼𝑛𝑐𝑜𝑚𝑒". 𝐿𝑜𝑤-𝐼𝑛𝑐𝑜𝑚𝑒" is an
indicator for being a low-income county defined as a county with its pci in 2013 lower than
the 2013 median pci. Medicaid is the public insurance program mainly for the low-income
population, so its impact on insurance coverage is expected to be larger in counties with lower
income levels. Thus, if the DID result is actually shaped by the treatment effect (rather than
state-specific trends), the DID coefficients are expected to be different between low-income
and high-income counties. What is your conclusion?

Question 5: Regression Discontinuity – US Senate Elections [45 points]
This exercise will measure the incumbency advantage in the US Senate elections using a
regression discontinuity (RD) design. The incumbency advantage is defined as an incumbent’s
advantage in an election over challengers. Identifying the incumbency advantage from election
results is not straightforward. High-quality politicians are more likely to win an election in general,
so it may be unclear whether the candidate’s quality or incumbency allows the incumbent to be re-
elected. Therefore, a simple comparison of the probability of being elected between incumbents
and new candidates may overestimate the incumbency advantage.
In this exercise, we focus on measuring the effect of being an incumbent party on the vote share.
Lee (2008) shows that using an RD design can effectively measure the advantage of an incumbent
party in the US House elections. He uses the vote margin of a party in the previous election as a
running variable (the party’s vote share – the opposite party’s vote share). In a two-party system
like the one in the US, having a positive vote margin means the party won the election. Thus, the
discontinuity in the running variable occurs at the zero-vote margin, and the treatment (being
elected) is defined as having a positive vote margin.
This exercise will focus on US Senate elections to measure the incumbency advantage. The dataset
is CIT_2019_Cambridge_senate.dta from Cattaneo, Frandsen, and Titiunik (2015). It contains the
results of the US Senate elections from 1914 to 2010. The running variable is demmv, which
measures the vote margin of the Democratic party in election t. The primary outcome variable of
interest is demvoteshfor2, Democratic vote share in election 𝑡 + 2. In US Senate elections, an
incumbent is up for re-election for the same seat not in the next election (𝑡 + 1) but in the election
after the next one (𝑡 + 2). Make sure to read Section 4 of Cattaneo, Frandsen, and Titiunik (2015)
to get a good grasp of the overall discussion of the US Senate elections and how the incumbency
advantage is defined in this setting.
The following website, https://rdpackages.github.io/, provides Stata and R codes you can use for
RD analyses.

a. Which RD design (Sharp RD vs. Fuzzy RD) is suitable for this study?

b. Draw the plots of demvoteshfor2 on demmv using the following features (for RD plots, use the fourth polynomial order for the global estimation).
i. A scatter plot using raw data. Include a vertical line for the cutoff.
ii. A Mimicking Variance (MV) RD plot with equally-spaced (ES) bins.
iii. An MV RD plot with quantile-spaced (QS) bins.
iv. An Integrated Mean Squared Error (IMSE) RD plot with ES bins
v. An IMSE RD plot with QS bins

c. What can you learn from b) – for example, by comparing the scatter plot and RD plots
in general, IMSE and MV RD plots, and ES and QS bins?

d. Estimate the RD treatment effect using the traditional approach. Specifically, use the
fourth-order polynomials for a global parametric estimation. Report the p-value and 95%
confidence interval for the RD coefficient using the heteroskedasticity-robust standard
error.

e. Estimate the RD treatment effect using the continuity-based approach. Specifically, use
a triangular kernel function and linear polynomial for estimation. Compute the Mean
Squared Error (MSE) optimal bandwidth. Report the p-values and 95% confidence
intervals for the RD coefficient using conventional and robust bias-correction standard
errors. How does your result with this approach differ from the result in d)? What do you
think are the reasons for the discrepancy?

f. Estimate the RD treatment effect using the local randomization approach. First, select
the window for local randomization using the following settings in the randomization-
based tests of the sharp null hypothesis of no treatment effect for each of the covariates
listed below.

i. The test statistic: the difference-in-means
ii. Covariates: State-level Democratic percentage of the vote in the past presidential election (predemvoteshlag1), state population (population), Democratic vote share
in Senate election 𝑡 − 1 (demvoteshlag1), Democratic vote share in Senate election 𝑡 − 2 (demvoteshlag2), an indicator for Democratic victory in Senate election 𝑡 − 1 (demwinprv1), an indicator for Democratic victory in Senate election 𝑡 − 2 (demwinprv2), an indicator for an open seat in election 𝑡 (dopen), an indicator for
midterm (non-presidential) election in election t (dmidterm), and an indicator for whether the president of the US at 𝑡 is Democratic (dpresdem)

iii. Randomization mechanism: complete (or fixed-margins) randomization
iv. Significance level (α∗) = 0.15
v. Conduct simulated-based tests with 10,000 draws, rather than exact tests, when
computing the p-values. See Chapter 2.2.1 in Cattaneo, Idorobo, and Titiunik
(2018) for more information about a simulated-based test.

Make a graph like Figure 1 in Cattaneo, Frandsen, and Titiunik (2015), which plots
minimum p-values against %
& (window length) for the window widths (i.e., 𝑥 values)
between 0 and 10 with an increment of 0.125. What is your final choice for the window?

g. Report the estimate of the RD effect from the local randomization approach with the
settings specified in f). Report the p-value and 95% confidence interval based on 10,000
simulations.

h. Conduct the following two falsification tests for the RD design (focusing on the continuity-based approach with non-parametric local-linear estimation).

i. Make an MV RD plot of demvoteshlag2 against the running variable, demmv, using QS bins. demvoteshlag2 is predetermined, so it is not expected to exhibit a discontinuity at the cutoff. Do you see any jump at the cutoff in your RD plot?

ii. Construct 95% robust bias-correction confidence intervals for the RD effect (demvoteshfor2) using the following placebo cutoffs: −6, −4, −2, 2, 4, and 6.
 Do you see any significant effects at the placebo cutoffs? Based on these results, what is your conclusion regarding the validation of the RD design?

i. Based on your results from the three approaches, what is your conclusion regarding the existence/magnitude of the incumbency advantage in US Senate elections?